# Homework - Week 01

## 01. Downloading the data

Download the data for January and February 2023.
Read the data for January. How many columns are there?

> 19

In [ ]:
import pandas as pd


# download the data for january 2023
df_january =  pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

# download the data for february 2023
df_february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')


In [5]:
# check the number of columns in the january data
print(f"Number of columns in January data: {len(df_january.columns)}")

Number of columns in January data: 19


## 02. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.
What's the standard deviation of the trips duration in January?

> 42.59


In [17]:
# create a new column for the trip duration
df_january['trip_duration'] = df_january['tpep_dropoff_datetime'] - df_january['tpep_pickup_datetime']

df_january.head()

# convert the trip duration to minutes
df_january['trip_duration'] = df_january['trip_duration'].dt.total_seconds() / 60


In [18]:
# compute the standard deviation of the trip distance in the january data
print(f"Standard deviation of trip duration in January data: {df_january['trip_duration'].std()}")

Standard deviation of trip duration in January data: 42.594351241920904


## 03. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%
92%
95%
98%

## 4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

2
155
345
515
715

## 05. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

3.64
7.64
11.64
16.64

## 06. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81
7.81
11.81
16.81